In [17]:
import paho.mqtt.client as mqtt
import json
from datetime import datetime
from kafka import KafkaProducer


mqtt_client= mqtt.Client()
producer = KafkaProducer(bootstrap_servers=["localhost:9092"], value_serializer=lambda x:json.dumps(x).encode("utf-8"))


def on_connect(client, userdata, flags, rc):
    if(str(rc) == '0'):
        print("mqtt connection successful...")
        mqtt_client.subscribe("iot")
    else:
        print("mqtt connection failed...")

def on_message(client, userdata, msg):
    try:
        payload = msg.payload.decode("utf-8")
        data = json.loads(payload)
        data['topic'] = msg.topic
        print(data)
        data['time'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(".", end="")
        producer.send(topic="iot", value=data)
    except Exception as ex:
        print(ex)

mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_message
mqtt_client.connect(host="broker.hivemq.com", port=1883)
mqtt_client.loop_start()

/tmp/ipykernel_109991/4100433970.py:7: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqtt_client= mqtt.Client()


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

mqtt connection successful...
{'proximitySensor': {'x': '5.0'}, 'temperatureSensor': {'temperature': '26.0'}, 'lightSensor': {'illuminance': '26.0'}, 'gyroscopeSensor': {'x': '0.0', 'y': '0.0', 'z': '0.0'}, 'gpsSensor': {'alt': '619.5', 'lon': '73.71', 'lat': '18.59'}, 'accelerometerSensor': {'x': '-0.4', 'y': '-0.15', 'z': '9.75'}, 'topic': 'iot'}
.{'proximitySensor': {'x': '5.0'}, 'temperatureSensor': {'temperature': '33.0'}, 'lightSensor': {'illuminance': '33.0'}, 'gyroscopeSensor': {'x': '0.0', 'y': '0.0', 'z': '0.0'}, 'gpsSensor': {'alt': '619.5', 'lon': '73.71', 'lat': '18.59'}, 'accelerometerSensor': {'x': '-0.39', 'y': '-0.16', 'z': '9.74'}, 'topic': 'iot'}
.{'proximitySensor': {'x': '5.0'}, 'temperatureSensor': {'temperature': '26.0'}, 'lightSensor': {'illuminance': '26.0'}, 'gyroscopeSensor': {'x': '0.0', 'y': '0.0', 'z': '0.0'}, 'gpsSensor': {'alt': '619.5', 'lon': '73.71', 'lat': '18.59'}, 'accelerometerSensor': {'x': '-0.4', 'y': '-0.15', 'z': '9.75'}, 'topic': 'iot'}
.{'p

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder\
            .appName("miniproject")\
            .config("spark.sql.shuffle.partitions", "4")\
            .master("local[*]")\
            .getOrCreate()

In [19]:
kafka_pub_topic = "iot"
data = spark.readStream\
            .format("kafka")\
            .option("kafka.bootstrap.servers", "localhost:9092")\
            .option("subscribe", kafka_pub_topic)\
            .option("startingOffsets", "latest")\
            .load()

data.printSchema()

AnalysisException:  Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".        